In [ ]:
# === Imports === #
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import yaml
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Load Configuration === #
def load_config(path):
    with open(path, "r") as file:
        return yaml.safe_load(file)

config = load_config(CONFIG_PATH)

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

# === Retrieve Account Information === #
account_info = mt5.account_info()
if account_info is None:
    print("Failed to retrieve account information.")
    mt5.shutdown()
    quit()

balance = account_info.balance
equity = account_info.equity + account_info.credit
floating_loss = account_info.profit

print("=== Account Information ===")
print(f"Balance: {balance:.2f} USD")
print(f"Equity: {equity:.2f} USD")
print(f"Floating Loss: {floating_loss:.2f} USD")

# === Load Running Trades from MT5 and Reference Google Sheet === #
fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
df = fetcher.get_sheet_as_dataframe("Forex")
positions = mt5.positions_get()
df_positions = pd.DataFrame([pos._asdict() for pos in positions]) if positions else pd.DataFrame()

if df_positions.empty:
    print("No running trades found. Error:", mt5.last_error())
    mt5.shutdown()
    quit()

# === Clean and Rename Trade Columns === #
df_positions = df_positions[["type", "volume", "price_open", "price_current", "sl", "tp", "profit", "symbol"]]
df_positions["type"] = df_positions["type"].replace({0: "Buy", 1: "Sell"})
df_positions.rename(columns={
    "symbol": "Symbol",
    "type": "Action",
    "volume": "Lot",
    "price_open": "Price",
    "price_current": "Price Current",
    "sl": "SL",
    "tp": "TP",
    "profit": "Profit"
}, inplace=True)

# Prepare hedge profit DataFrame early for printing
df_trend = df[[df.columns[0], df.columns[1], df.columns[2], df.columns[19]]].drop_duplicates()
df_trend.columns = ["Symbol", "Type", "Action", "Trend"]

df_temp_profit = df_positions[["Symbol", "Action", "Profit"]]
df_hedge = pd.merge(df_trend, df_temp_profit, on=["Symbol", "Action"], how="left")
hedged_profit = df_hedge.loc[
    (df_hedge["Type"] == "Forex") & (df_hedge["Trend"] == "Sideways"),
    "Profit"
].sum()

print(f"Total hedged amount (Forex, Sideways trend): {hedged_profit} USD")

# === Aggregate Positions by Symbol and Action === #
def aggregate_group(group):
    symbol = group["Symbol"].iloc[0]
    action = group["Action"].iloc[0]
    lot = group["Lot"].sum()
    warnings = []

    df_lot_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[5]].sum()
    if not pd.isna(df_lot_value) and round(df_lot_value, 2) > round(lot, 2):
        warnings.append(f"Lot Mismatch: Expected {round(df_lot_value, 2)}, Found {round(lot, 2)}")

    df_sl_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[7]].sum()
    sl_mode = group["SL"].mode().iloc[0] if not group["SL"].mode().empty else np.nan
    if group["SL"].nunique() > 1 or (df_sl_value != 0 and round(df_sl_value, 2) != round(sl_mode, 2)):
        warnings.append("SL Mismatch")

    df_tp_value = df.loc[(df.iloc[:, 0] == symbol) & (df.iloc[:, 2] == action), df.columns[8]].sum()
    tp_mode = group["TP"].mode().iloc[0] if not group["TP"].mode().empty else np.nan
    if group["TP"].nunique() > 1 or round(df_tp_value, 2) != round(tp_mode, 2):
        warnings.append("TP Mismatch")

    warning_message = "; ".join(warnings) if warnings else np.nan

    return pd.Series({
        "Lot": lot,
        "Price": (group["Price"] * group["Lot"]).sum() / group["Lot"].sum(),
        "Price Current": group["Price Current"].mean(),
        "SL": sl_mode,
        "TP": tp_mode,
        "Profit": group["Profit"].sum(),
        "Warning": warning_message
    })

df_positions = (
    df_positions.groupby(["Symbol", "Action"], as_index=False)
    .apply(aggregate_group)
    .reset_index(drop=True)
)

# === Shutdown MT5 === #
mt5.shutdown()

# === Calculations and Merging === #
df_positions["Coeff"] = np.abs(df_positions["Profit"] / (df_positions["Price Current"] - df_positions["Price"])) / (df_positions["Lot"] * 100)
df_positions["Pips"] = np.abs(df_positions["TP"] - df_positions["SL"])

df_type = df[[df.columns[0], df.columns[1]]].drop_duplicates()
df_type.columns = ["Symbol", "Type"]
df_positions = pd.merge(df_positions, df_type, on="Symbol", how="left")
df_positions["Type"].fillna("Unknown", inplace=True)

# === Update Coefficients in Config File === #
df_avg_coeff = df_positions.groupby("Symbol")["Coeff"].mean().reset_index()
differences = {}
for _, row in df_avg_coeff.iterrows():
    symbol = row["Symbol"]
    avg_coeff = round(row["Coeff"], 5)
    if not pd.isna(avg_coeff):
        if symbol in config.get("symbol_coefficients", {}):
            old_coeff = config["symbol_coefficients"].get(symbol, avg_coeff)
            if old_coeff != avg_coeff:
                differences[symbol] = {"Old": old_coeff, "New": avg_coeff}
        config["symbol_coefficients"][symbol] = avg_coeff

with open(CONFIG_PATH, "w") as file:
    yaml.safe_dump(config, file)

print("Changed symbol_coefficients:")
for symbol, change in differences.items():
    print(f"{symbol}: {change['Old']} -> {change['New']}")

# === Add Account Summary === #
df_balance = pd.DataFrame([
    {"Symbol": "Balance", "Price": balance},
    {"Symbol": "Equity", "Price": equity},
    {"Symbol": "Floating Loss", "Price": floating_loss}
])

df_final = pd.concat([df_balance, df_positions], ignore_index=True)

# === Upload to Google Sheets === #
uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
uploader.upload_dataframe(df_final, "Forex Summary", replace=True)

# === Print Final Output === #
print("Final Merged DataFrame:")
df_final

=== Account Information ===
Balance: 7068.02 USD
Equity: 7704.78 USD
Floating Loss: -3065.10 USD


In [ ]:
import time
import MetaTrader5 as mt5
import pandas as pd
import os
from google_sheet_api import GoogleSheetsUploader

# === Setup Paths === #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")

# === Initialize MT5 Connection === #
if not mt5.initialize():
    print("MT5 initialization failed.")
    quit()

fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")

# Load DataFrame from Google Sheets
df = fetcher.get_sheet_as_dataframe("Forex")
df.columns = df.iloc[6]
df = df.iloc[7:-3].reset_index(drop=True)

# Initialize counters for success and failure
success_count = 0
failure_count = 0

# Fetch unique symbols from running trades in MT5
positions = mt5.positions_get()
if positions is None or len(positions) == 0:
    print("No running trades found. Exiting.")
    mt5.shutdown()
    quit()

# Get unique symbols from MT5 positions
symbol_list = sorted(list(set(pos.symbol for pos in positions)))

# Convert positions to DataFrame for easy filtering
df_positions = pd.DataFrame([pos._asdict() for pos in positions])

# Iterate through each unique symbol from MT5 positions
for symbol in symbol_list:
    print(f"\n=== Checking Symbol: {symbol} ===")
    for action in ["Buy", "Sell"]:
        print(f"  Action: {action}")
        action_type = 0 if action == "Buy" else 1
        filtered_positions = df_positions[(df_positions['symbol'] == symbol) & (df_positions['type'] == action_type)]
        
        if filtered_positions.empty:
            print(f"    ⚠️ No {action} positions running. Skipping.")
            continue

        # Filter rows for the current symbol and action from Google Sheets
        df_filtered = df[(df["Symbol"] == symbol) & (df["Action"] == action)]
        
        if df_filtered.empty:
            print(f"    ⚠️ No matching TP/SL settings in Google Sheets.")
            continue
        
        tp_value = round(float(df_filtered['TP'].values[0]), df_filtered['Decimal'].values[0])
        sl_value = round(float(df_filtered['SL'].values[0]), df_filtered['Decimal'].values[0])

        # Iterate over filtered positions for modification
        for _, pos in filtered_positions.iterrows():
            order_id = pos['ticket']
            current_tp = round(pos['tp'], df_filtered['Decimal'].values[0])
            current_sl = round(pos['sl'], df_filtered['Decimal'].values[0])

            # Check for differences and print
            print(f"    Position ID: {order_id}")
            print(f"    Current TP: {current_tp}, Current SL: {current_sl}")
            print(f"    Target TP: {tp_value}, Target SL: {sl_value}")

            if current_tp == tp_value and current_sl == sl_value:
                print(f"    ✅ No changes needed for {action} position {order_id}.")
                continue
            else:
                print(f"    ⚠️ TP/SL difference detected. Attempting to update...")

            # Modify TP/SL using position ID
            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": order_id,
                "sl": sl_value,
                "tp": tp_value
            }

            # Send modification request
            result = mt5.order_send(request)

            # Check if result is None (Error)
            if result is None:
                print(f"    ⚠️ Failed to modify {action} position {order_id}. Error: Request returned None.")
                failure_count += 1
                continue

            # Check if the modification was successful
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"    ✅ {action} position {order_id} modified successfully.")
                print(f"    Before: TP={current_tp}, SL={current_sl}")
                print(f"    After:  TP={tp_value}, SL={sl_value}\n")
                success_count += 1
            else:
                print(f"    ⚠️ Failed to modify {action} position {order_id}. Error: {result.comment}")
                failure_count += 1

# === Shutdown MT5 Connection === #
mt5.shutdown()

# === Summary Report ===
print("\n=== TP/SL Modification Summary ===")
print(f"Total Successful Modifications: {success_count}")
print(f"Total Failed Modifications: {failure_count}")

✅ Successfully retrieved data from 'Forex' as a DataFrame.

=== Checking Symbol: AAPL.NAS ===
  Action: Buy
    Position ID: 3297549570
    Current TP: 217.47, Current SL: 112.36
    Target TP: 217.47, Target SL: 112.36
    ✅ No changes needed for Buy position 3297549570.
  Action: Sell
    ⚠️ No Sell positions running. Skipping.

=== Checking Symbol: AAVEUSD ===
  Action: Buy
    Position ID: 3319810775
    Current TP: 338.66, Current SL: 0.01
    Target TP: 338.66, Target SL: 0.01
    ✅ No changes needed for Buy position 3319810775.
  Action: Sell
    ⚠️ No Sell positions running. Skipping.

=== Checking Symbol: ABBV.NYSE ===
  Action: Buy
    Position ID: 3298446412
    Current TP: 200.55, Current SL: 142.89
    Target TP: 200.55, Target SL: 142.89
    ✅ No changes needed for Buy position 3298446412.
  Action: Sell
    ⚠️ No Sell positions running. Skipping.

=== Checking Symbol: ABNB.NAS ===
  Action: Buy
    Position ID: 3318618882
    Current TP: 143.89, Current SL: 0.01
    Targ